In [1]:
!pip install psycopg2

In [5]:
import psycopg2
from datetime import date
from typing import List

def start_planning(year, quarter, user, pwd):
    con = psycopg2.connect(database='2020_plan_LSharaborina', user=user, port='5433', password=pwd, host='localhost')
    cur = con.cursor()
    cur.execute(""" DELETE 
                    FROM plan_data p
                    WHERE p.quarterid = '%s.%s'
                """, [year, quarter])
    cur.execute(""" DELETE 
                    FROM plan_status p
                    WHERE p.quarterid = '%s.%s'
                """, [year, quarter])
    cur.execute(""" INSERT into plan_status (quarterid, country, status, modifieddatetime, author)
                    SELECT '%s.%s', country2.countrycode, 'R', current_timestamp, %s
                    FROM country2
                """, [year, quarter, user])
    for status in ['N', 'P']:
        cur.execute(""" INSERT into plan_data (versionid, country, quarterid, pcid, salesamt)
                SELECT %s, country2.countrycode, '%s.%s', company_sales.categoryid, avg(COALESCE(company_sales.salesamt, 0))
                FROM country2
                INNER JOIN company on country2.countrycode = company.countrycode
                INNER JOIN company_sales on company.id = company_sales.cid
                where company_sales.qr = '%s.%s'
                GROUP BY 2, 4
                """, [status, year, quarter, year, quarter])
    
# #     cur.itersize = 1
#     for record in cur:
# #         print(record[0], record[1], record[2])
#         print(record)
#     record = cur.fetchone()
#     while record:
#         print(record)
#         record = cur.fetchone()
    con.commit()
    con.close()

start_planning(2014, 1, 'ivan', 'sql3')

